In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import joblib

# Load the dataset
df = pd.read_csv("pg_coliving_dataset.csv")

# Keep only the listed columns
columns = [
    'listing_title', 'city', 'area', 'location', 'zone', 'shearing', 'best_suit_for',
    'meals_avilable', 'notic_period', 'lock_in_period', 'amenities_count',
    'in_time_at_night', 'non_veg_allowed', 'opposite_gender_allowed',
    'visitors_allowed', 'gurdian_allowed', 'drinking_allowed', 'smoking_allowed',
    'security_deposite', 'rent_price'
]
df = df[columns].copy()

#  Handle numeric columns
df['amenities_count'] = pd.to_numeric(df['amenities_count'], errors='coerce').fillna(0)
df['security_deposite'] = pd.to_numeric(df['security_deposite'], errors='coerce').fillna(0)
df['rent_price'] = pd.to_numeric(df['rent_price'], errors='coerce')

#  Drop missing target values
df.dropna(subset=['rent_price'], inplace=True)

#  Split features and target
X = df.drop(columns='rent_price')
y = df['rent_price']

#  Identify categorical columns (exclude numeric ones)
cat_cols = X.select_dtypes(include='object').columns.tolist()

#  Preprocessing and model pipeline
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
], remainder='passthrough')

model = Pipeline([
    ('pre', preprocessor),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
])

#  Train/test split and model training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

#  Save the trained model
joblib.dump(model, "pg_rent_model_full.pkl")
print(" Model saved as pg_rent_model_full.pkl")


 Model saved as pg_rent_model_full.pkl


In [2]:
import pandas as pd
import joblib

def predict_pg_rent_full(
    listing_title, city, area, location, zone, shearing, best_suit_for,
    meals_avilable, notic_period, lock_in_period, amenities_count,
    in_time_at_night, non_veg_allowed, opposite_gender_allowed,
    visitors_allowed, gurdian_allowed, drinking_allowed, smoking_allowed,
    security_deposite
):
    # Load model
    model = joblib.load("pg_rent_model_full.pkl")

    # Create input dataframe
    input_df = pd.DataFrame([{
        'listing_title': listing_title,
        'city': city,
        'area': area,
        'location': location,
        'zone': zone,
        'shearing': shearing,
        'best_suit_for': best_suit_for,
        'meals_avilable': meals_avilable,
        'notic_period': notic_period,
        'lock_in_period': lock_in_period,
        'amenities_count': amenities_count,
        'in_time_at_night': in_time_at_night,
        'non_veg_allowed': non_veg_allowed,
        'opposite_gender_allowed': opposite_gender_allowed,
        'visitors_allowed': visitors_allowed,
        'gurdian_allowed': gurdian_allowed,
        'drinking_allowed': drinking_allowed,
        'smoking_allowed': smoking_allowed,
        'security_deposite': security_deposite
    }])

    # Predict
    pred_rent = model.predict(input_df)[0]
    return round(pred_rent, 2)


In [3]:
rent = predict_pg_rent_full(
    listing_title="pg for boys",
    city="Nagpur",
    area="Mankapur", 
    location="Near Bus Stop",
    zone="North",
    shearing="tripal",
    best_suit_for="Students", 
    meals_avilable="Yes",
    notic_period="15 days",
    lock_in_period="3 months", 
    amenities_count=6,
    in_time_at_night="11 PM",
    non_veg_allowed="Yes",
    opposite_gender_allowed="No",
    visitors_allowed="Yes", 
    gurdian_allowed="Yes", 
    drinking_allowed="No",
    smoking_allowed="No", 
    security_deposite=3000
)
print(f"Predicted Rent: ₹{rent}")


Predicted Rent: ₹4711.5


In [4]:
rent = predict_pg_rent_full(
    listing_title="pg for girls",
    city="Nagpur",
    area="jaitala", 
    location="jaitala, nagpur",
    zone="south",
    shearing="private",
    best_suit_for="Students", 
    meals_avilable="Yes",
    notic_period="15 days",
    lock_in_period="3 months", 
    amenities_count=7,
    in_time_at_night="10 PM",
    non_veg_allowed="Yes",
    opposite_gender_allowed="No",
    visitors_allowed="Yes", 
    gurdian_allowed="Yes", 
    drinking_allowed="No",
    smoking_allowed="No", 
    security_deposite=3000
)
print(f"Predicted Rent: ₹{rent}")


Predicted Rent: ₹8147.0


In [6]:
rent = predict_pg_rent_full(
    listing_title="pg for girl and boys",
    city="Nagpur",
    area="omkar nagar", 
    location="omkar nagar, nagpur",
    zone="south",
    shearing="private",
    best_suit_for="Students, woring", 
    meals_avilable="Yes",
    notic_period="10 days",
    lock_in_period="2 months", 
    amenities_count=5,
    in_time_at_night="no",
    non_veg_allowed="Yes",
    opposite_gender_allowed="yes",
    visitors_allowed="Yes", 
    gurdian_allowed="Yes", 
    drinking_allowed="No",
    smoking_allowed="No", 
    security_deposite=4000
)
print(f"Predicted Rent: ₹{rent}")


Predicted Rent: ₹8390.0
